# gurobipy 8.0 breaks back compatibility with float("inf")

The `gurobipy` package has broken long standing `inf`  functionality with their 8.0 release. Historically (dating back at least 5 years) `inf` is replaced with `GRB.INFINITY` during model building.  The 8.0 release instead raises an exception upon encountering `inf`.

To demonstrate this functionality, we will use a diet.xls file that has infinity as part of the input data. Specifically, this file specifies an unbounded upper limit for protein consumption. It is consistent with the standard Gurobi example data set. 

You can download this diet.xls file from [here](https://drive.google.com/drive/folders/18w1hwJDrh4RJWwJi2tZb3qVUisUT5Q2k?usp=sharing). 

In [1]:
import os
assert os.path.isdir("data"), "you need a data directory to hold the data.xls file"
assert os.path.isfile(os.path.join("data", "diet.xls")), "need the data file"

We will be using the diet4.py file you can get from [here](http://www.gurobi.com/documentation/7.5/examples/diet4_py.html) and also the dietmodel.py file you can get from [here](http://www.gurobi.com/documentation/7.5/examples/dietmodel_py.html).

In [2]:
assert os.path.isdir("gurobi_code"), "you need a gurobi_code directory to hold the gurobi example code"
assert os.path.isfile(os.path.join("gurobi_code", "diet4.py")), "need the diet4.py example file"
assert os.path.isfile(os.path.join("gurobi_code", "dietmodel.py")), "need the diet4.py example file"

For Gurobi 7.5 (and all previous `gurobipy` versions I am aware of), this model behaves in the expected fashion. First it finds a solution of 11.8289 and then the altered model is determined to be infeasible.

If you run this with Gurobi 8.0 it throws an exception. Feel free to rerun the notebook on your own installation.

In [6]:
import gurobipy as gu

In [7]:
gu.GRB.VERSION_MAJOR, gu.GRB.VERSION_MINOR, gu.GRB.VERSION_TECHNICAL

(7, 5, 2)

In [4]:
cd gurobi_code

/Users/XXXXX/2979/gurobi_code


In [5]:
 run diet4.py

Optimize a model with 4 rows, 12 columns and 39 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [9e-01, 3e+00]
  Bounds range     [6e+01, 2e+03]
  RHS range        [6e+01, 2e+03]
Presolve removed 0 rows and 2 columns
Presolve time: 0.00s
Presolved: 4 rows, 10 columns, 37 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.472500e+02   0.000000e+00      0s
       4    1.1828861e+01   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds
Optimal objective  1.182886111e+01

Cost: 11.8289

Buy:
hamburger 0.604514
ice cream 2.59132
milk 6.97014

Adding constraint: at most 6 servings of dairy
Optimize a model with 5 rows, 12 columns and 41 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [9e-01, 3e+00]
  Bounds range     [6e+01, 2e+03]
  RHS range        [6e+00, 2e+03]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    

This example involves reading data from an .xls file with the `xlrd` package. The same sort of problem can be demonstrated using the widely popular `pandas` package reading from `.xlsx`, `.csv` or SQLite files (or AFAIK, from nearly every file format `pandas` supports. See Jeff Reback's comment re: infinity support [here](https://github.com/pandas-dev/pandas/issues/10065)).

Note that you can't determine if your engine is at risk of this exception failure simply by scanning your code for explicit references to `float("inf")`. Since `inf` is a fundamental part of Python, any of the packages you are importing might themselves be introducing `inf` just as `xlrd` did in this example. I am not aware of a turn-key solution. 